# CSE 158 FA20: Assignment 1
Name: Cameron Thomas <br>
PID: A13978010

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import gzip
import random
import scipy
from collections import defaultdict
from sklearn.linear_model import LogisticRegression
import json
import string
from nltk.stem.porter import PorterStemmer
from sklearn import linear_model
import heapq
import math
from sklearn.svm import SVC
from tqdm import tqdm
from tqdm.notebook import trange, tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import pdist, squareform

tar xvzf assignment1.tar.gz

In [2]:
os.listdir('assignment1')

['train.json.gz',
 'pairs_Category.txt',
 'test_Category.json.gz',
 'train_Category.json.gz',
 'pairs_Played.txt',
 'baselines.py',
 'pairs_Hours.txt']

**Play prediction (both classes)** Predict given a (user,game) pair from ‘pairs Played.txt’ whether the user
would play the game (0 or 1). Accuracy will be measured in terms of the categorization accuracy (fraction
of correct predictions). The test set has been constructed such that exactly 50% of the pairs correspond
to played games and the other 50% do not.

In [3]:
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

dataset = list(readGz("assignment1/train.json.gz"))

In [4]:
dataset[0]

{'hours': 0.3,
 'gameID': 'b96045472',
 'hours_transformed': 0.37851162325372983,
 'early_access': False,
 'date': '2015-04-08',
 'text': '+1',
 'userID': 'u01561183'}

In [5]:
dataset[1]

{'userID': 'u88836191',
 'early_access': False,
 'hours': 63.5,
 'hours_transformed': 6.011227255423254,
 'found_funny': 1,
 'text': 'If you want to sit in queue for 10-20min and have 140 ping then this game is perfect for you :)',
 'gameID': 'b19457938',
 'user_id': '76561198030408772',
 'date': '2017-05-20'}

In [6]:
len(dataset)

175000

In [7]:
usersPerGame = defaultdict(set)
GamesPerUser = defaultdict(set)
games = set()
for d in dataset:
    u,g = d['userID'], d['gameID']
    usersPerGame[g].add(u)
    GamesPerUser[u].add(g)
    if (g not in games):
        games.add(g)

In [8]:
X_train = dataset[:165000]
X_valid = dataset[165000:]

In [9]:
gameCount = defaultdict(int)
totalPlayed = 0

for d in dataset:
    gameCount[d['gameID']] += 1
    totalPlayed += 1

mostPopular = [(gameCount[x], x) for x in gameCount]
mostPopular.sort()
mostPopular.reverse()

In [10]:
def popularity(game):
    count = 0
    for g in mostPopular:
        if g[1] != game:
            count = count + g[0]
        else:
            count = count + g[0]
            break
    #print(count/totalPlayed)
    return (count/totalPlayed)

In [16]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom > 0:
        return numer/denom
    return 0

In [17]:
def Jaccard_score(u, g):
    similarities = []
    users = set(usersPerGame[g])
    for g2 in GamesPerUser[u]:
        sim = Jaccard(users,set(usersPerGame[g2]))
        if sim > 0:
            similarities.append(sim)
    similarities.sort()
    if len(similarities) == 0: return 0
    average = np.mean(similarities)
    return average

In [18]:
def cos_sim(a,b):
    numer = len(a.intersection(b))
    denom = len(a) * len(b)
    if denom > 0:
        return numer/denom
    return 0

In [19]:
def cos_score(u,g):
    similarities = []
    uprime = usersPerGame[g]
    for i in uprime:
        game = GamesPerUser[i]
        candidateItems = u
        sim = cos_sim(game, GamesPerUser[candidateItems])
        if sim > 0:
            similarities.append(sim)
    similarities.sort()
    #print(len(similarities))
    if len(similarities) == 0: return 0
    return max(similarities)

In [20]:
neg = []

for i in X_valid:
    r = {'userID': 0, 'gameID': 0, 'played': 0}
    r['userID'] = i['userID']
    game = random.sample(games,1)[0]
    while (game in GamesPerUser[r['userID']]):
        game = random.sample(games,1)[0]
    r['gameID'] = game
    neg.append(r)


pos = [{'userID': i['userID'], 'gameID': i['gameID'], 'played':1} for i in X_valid]
mix = pos + neg

In [21]:
usersPerGame = defaultdict(set)
GamesPerUser = defaultdict(set)
games = set()
for d in X_train:
    u,g = d['userID'], d['gameID']
    usersPerGame[g].add(u)
    GamesPerUser[u].add(g)
    if (g not in games):
        games.add(g)

In [22]:
GamesPerUserValid = defaultdict(set)
for d in mix:
    u,g = d['userID'], d['gameID']
    GamesPerUserValid[u].add(g)

In [23]:
GamesPerUserValidScore = defaultdict(set)
for u in tqdm(GamesPerUserValid):
    for g in GamesPerUserValid[u]:
        pop = popularity(g)
        jac = Jaccard_score(u,g)
        cos = cos_score(u,g)
        dp = (jac * cos) / pop
        GamesPerUserValidScore[u].add((dp, g))

In [24]:
GamesPerUserValidScore

defaultdict(set,
            {'u49969792': {(4.2927146975996705e-05, 'b34179104'),
              (0.0019078453111845105, 'b25961467')},
             'u33147591': {(1.2158029974109574e-05, 'b23088564'),
              (5.6412872248348915e-05, 'b15831593'),
              (0.00016346961324662816, 'b73229067'),
              (0.0005959377222910264, 'b20515626')},
             'u00954406': {(2.817370444182954e-05, 'b37571125'),
              (3.6538587778653144e-05, 'b31692548'),
              (4.2217666370962336e-05, 'b38961211'),
              (5.4216580681630876e-05, 'b62854383'),
              (5.644847462593709e-05, 'b98132988'),
              (9.321210593435867e-05, 'b37068085')},
             'u40416473': {(2.2410970186180302e-05, 'b74562793'),
              (0.0001168729550350153, 'b85572219')},
             'u08125051': {(3.4489923398379564e-05, 'b84011319'),
              (3.86423780287018e-05, 'b72312544'),
              (4.81473698292957e-05, 'b25299351'),
              (4.886314

In [25]:
rank_valid = []
for u in GamesPerUserValidScore:
    GamesPerUserValidScore[u] = sorted(GamesPerUserValidScore[u], reverse = True)
    length = math.ceil(len(GamesPerUserValidScore[u])/2)
    count = 0
    for pair in GamesPerUserValidScore[u]:
        if count != length:
            rank_valid.append((u,pair[1]))
            count = count + 1
        else:
            break

In [26]:
rank_valid

[('u49969792', 'b25961467'),
 ('u33147591', 'b20515626'),
 ('u33147591', 'b73229067'),
 ('u00954406', 'b37068085'),
 ('u00954406', 'b98132988'),
 ('u00954406', 'b62854383'),
 ('u40416473', 'b85572219'),
 ('u08125051', 'b88846011'),
 ('u08125051', 'b17552435'),
 ('u08125051', 'b01049948'),
 ('u16794935', 'b19939295'),
 ('u16794935', 'b50099026'),
 ('u16794935', 'b24539721'),
 ('u42345370', 'b37201339'),
 ('u40134397', 'b22332890'),
 ('u38053603', 'b81482141'),
 ('u38053603', 'b44539786'),
 ('u25257262', 'b53864614'),
 ('u25257262', 'b64105731'),
 ('u04355033', 'b27229867'),
 ('u04355033', 'b67179888'),
 ('u40085824', 'b58097218'),
 ('u40085824', 'b75479968'),
 ('u66137217', 'b95263410'),
 ('u66137217', 'b24479894'),
 ('u66137217', 'b20439501'),
 ('u66137217', 'b72051448'),
 ('u66137217', 'b26822015'),
 ('u66137217', 'b93550286'),
 ('u66137217', 'b23181479'),
 ('u66137217', 'b05546112'),
 ('u60516559', 'b82987255'),
 ('u60516559', 'b92840560'),
 ('u60516559', 'b29261461'),
 ('u79477484',

In [23]:
validateSet = []
trueSet = []

for i in mix:
    if ((i['userID'], i['gameID']) in rank_valid):
        validateSet.append(True)
    else:
        validateSet.append(False)
        
    if (i['played'] == 1):
        trueSet.append(True)
    else:
        trueSet.append(False)

In [24]:
zipping = list(zip(validateSet, trueSet))
acc = [i[0] == i[1] for i in zipping]
accuracy = sum(acc)/len(acc)
print("accuracy = " + str(accuracy))

accuracy = 0.74085


In [26]:
test_data = []
for l in open("assignment1/pairs_Played.txt"):
    if l.startswith("userID"):
        continue
    u,b = l.strip().split('-')
    test_data.append((u,b))

In [27]:
GamesPerUserTest = defaultdict(set)
for d in test_data:
    u,g = d[0], d[1]
    GamesPerUserTest[u].add(g)

In [28]:
GamesPerUserScore = defaultdict(set)
for u in GamesPerUserTest:
    for g in GamesPerUserTest[u]:
        pop = popularity(g)
        jac = Jaccard_score(u,g)
        cos = cos_score(u,g)
        dp = (jac * cos) / pop
        GamesPerUserScore[u].add((dp, g))

In [29]:
rank = []
for u in GamesPerUserScore:
    GamesPerUserScore[u] = sorted(GamesPerUserScore[u], reverse = True)
    length = math.ceil(len(GamesPerUserScore[u])/2)
    count = 0
    for pair in GamesPerUserScore[u]:
        if count != length:
            rank.append((u,pair[1]))
            count = count + 1
        else:
            break

In [30]:
predictions = open("predictions_Played2.txt", 'w')
for l in open("assignment1/pairs_Played.txt"):
    if l.startswith("userID"):
    #header
        predictions.write(l)
        continue
    u,g = l.strip().split('-')
    if (u,g) in rank:
        predictions.write(u + '-' + g + ",1\n")
    else:
        predictions.write(u + '-' + g + ",0\n")
        
predictions.close()